# Imports

In [1]:
# add to sys.path
import sys

sys.path.insert(0, "../")
sys.path.insert(0, "../src")

In [2]:
import time
import logging
import pandas as pd
from datetime import datetime, timedelta, timezone
from dateutil.relativedelta import relativedelta
from google.cloud import bigquery
import plaid

# SCHEMAS
from schemas.bq_table_schemas import BqTableSchemas
from schemas.cloud_schemas import CloudSchemas


# UTILS
from utils.bq_utils import BqUtils
from utils.gcp_utils import GcpUtils
from utils.plaid_utils import PlaidUtils
from utils.secrets_utils import SecretsUtils
from utils.crypto_utils import CryptoUtils
from utils.budget_values import BudgetValues
from jobs.financial_accounts import FinancialAccounts
from jobs.plaid_transactions import PlaidTransactions
from jobs.plaid_investments import PlaidInvestments


# get all secrets
sec = SecretsUtils()
secrets = sec.create_secrets_dict(plaid_secret_type="PROD")
PLAID_CLIENT_ID = secrets["PLAID_CLIENT_ID"]
PLAID_SECRET = secrets["PLAID_SECRET_PROD"]
PLAID_ACCESS_TOKENS = sec.get_access_token_secrets(secrets)
PLAID_HOST = plaid.Environment.Production
PLAID_PRODUCTS = ["liabilities", "transactions", "investments"]
PLAID_COUNTRY_CODES = ["US"]
CRYPTO_SECRETS = secrets["CRYPTO_SECRETS"]

# initialize clients
bq_client = bigquery.Client()
bq = BqUtils(bq_client=bq_client)
plaid_client = PlaidUtils(bq_client, PLAID_CLIENT_ID, PLAID_SECRET, PLAID_HOST)
financial_accounts = FinancialAccounts(bq_client, plaid_client)
plaid_transactions = PlaidTransactions(bq_client, plaid_client)
plaid_investments = PlaidInvestments(bq_client, plaid_client)
budget_values = BudgetValues(bq_client)
crypto = CryptoUtils()
bq_tables = BqTableSchemas()
gcp = GcpUtils(bq_client=bq_client)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 1000)

# Run Budget Values

In [3]:
offset = bq.get_date_offset(date_str="2022-10", partition_format="YYYYMM")
print(offset)

for i in range(offset, 1):
    # print(bq.get_partition_date(i, partition_format="YYYYMM"))
    budget_values.upload_budget_values_df_to_bq(offset=i)

-22
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202210`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202211`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202212`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202301`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202302`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202303`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202304`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202305`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202306`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202307`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202308`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202309`
SUCCESS: df uploaded to `zsc-personal.budget_values.budget_values_202310`
SUCCESS: df uploaded to `zsc-perso

# Budget Values Backfill Analysis

In [4]:
trans_df = bq.query("""
SELECT 
    DATE_TRUNC(transaction_date, MONTH) AS transaction_month,
    category_raw,
    subcategory_raw,
    category_updated,
    subcategory_updated,
    category,
    subcategory,
    ROUND(SUM(actual_amount), 3) AS amount
FROM `zsc-personal.personal_finance.personal_finance_tableau_20240811`
WHERE metric_category = "TRANSACTIONS"
GROUP BY 1,2,3,4,5,6,7
""")



In [5]:
# calculate median
trans_df['median_per_category'] = trans_df.groupby(["category_raw", "subcategory_raw", "category_updated", "subcategory_updated", "category", "subcategory", ])['amount'].transform('median')

# sort df
trans_df.sort_values(by=["subcategory_updated", "transaction_month"], inplace=True)

# create category list to iterate 
categories = list(trans_df["subcategory_updated"].unique())

print(trans_df.shape)
len(categories)

(579, 9)


67

In [6]:
for category in categories:
    print(category)
    
    # Make a copy of the sliced DataFrame to avoid SettingWithCopyWarning
    category_df = trans_df[trans_df["subcategory_updated"] == category].copy()

    category_df['transaction_month'] = category_df['transaction_month'].astype(str)
    
    
    amount_dict = category_df.set_index('transaction_month')['amount'].to_dict()
    median_dict = category_df.set_index('transaction_month')['median_per_category'].to_dict()

    display(category_df)
    
    print("ACTUAL AMOUNT")
    print(amount_dict)

    print("\nMEDIAN AMOUNT")
    print(median_dict)
    
    print()

BANK_FEES_ATM_FEES


,transaction_month,category_raw,subcategory_raw,category_updated,subcategory_updated,category,subcategory,amount,median_per_category
366,2023-03-01,BANK_FEES,BANK_FEES_ATM_FEES,BANK_FEES,BANK_FEES_ATM_FEES,Personal Spending,Bank Fees,-2.5,-3.75
365,2023-05-01,BANK_FEES,BANK_FEES_ATM_FEES,BANK_FEES,BANK_FEES_ATM_FEES,Personal Spending,Bank Fees,-5.0,-3.75


ACTUAL AMOUNT
{'2023-03-01': -2.5, '2023-05-01': -5.0}

MEDIAN AMOUNT
{'2023-03-01': -3.75, '2023-05-01': -3.75}

BANK_FEES_OTHER_BANK_FEES


,transaction_month,category_raw,subcategory_raw,category_updated,subcategory_updated,category,subcategory,amount,median_per_category
340,2023-12-01,BANK_FEES,BANK_FEES_OTHER_BANK_FEES,BANK_FEES,BANK_FEES_OTHER_BANK_FEES,Personal Spending,Bank Fees,-2.5,-2.5
341,2024-06-01,BANK_FEES,BANK_FEES_OTHER_BANK_FEES,BANK_FEES,BANK_FEES_OTHER_BANK_FEES,Personal Spending,Bank Fees,-2.5,-2.5


ACTUAL AMOUNT
{'2023-12-01': -2.5, '2024-06-01': -2.5}

MEDIAN AMOUNT
{'2023-12-01': -2.5, '2024-06-01': -2.5}

ENTERTAINMENT_CASINOS_AND_GAMBLING


...

ACTUAL AMOUNT
{'2023-03-01': -315.81, '2023-05-01': -27.0, '2023-07-01': -392.8, '2023-12-01': -578.38, '2024-03-01': -61.2, '2024-06-01': -415.73, '2024-07-01': -180.39, '2024-08-01': -80.0}

MEDIAN AMOUNT
{'2023-03-01': -248.1, '2023-05-01': -248.1, '2023-07-01': -248.1, '2023-12-01': -248.1, '2024-03-01': -248.1, '2024-06-01': -248.1, '2024-07-01': -248.1, '2024-08-01': -248.1}

TRAVEL_LODGING


,transaction_month,category_raw,subcategory_raw,category_updated,subcategory_updated,category,subcategory,amount,median_per_category
331,2023-04-01,TRAVEL,TRAVEL_LODGING,TRAVEL,TRAVEL_LODGING,Personal Spending,Travel,-473.93,-103.515
336,2023-05-01,TRAVEL,TRAVEL_LODGING,TRAVEL,TRAVEL_LODGING,Personal Spending,Travel,-2.21,-103.515
332,2023-07-01,TRAVEL,TRAVEL_LODGING,TRAVEL,TRAVEL_LODGING,Personal Spending,Travel,-122.03,-103.515
337,2023-11-01,TRAVEL,TRAVEL_LODGING,TRAVEL,TRAVEL_LODGING,Personal Spending,Travel,-75.00,-103.515
335,2024-01-01,TRAVEL,TRAVEL_LODGING,TRAVEL,TRAVEL_LODGING,Personal Spending,Travel,-85.00,-103.515
334,2024-02-01,TRAVEL,TRAVEL_LODGING,TRAVEL,TRAVEL_LODGING,Personal Spending,Travel,-299.22,-103.515
330,2024-05-01,TRAVEL,TRAVEL_LODGING,TRAVEL,TRAVEL_LODGING,Personal Spending,Travel,-3605.55,-103.515
333,2024-07-01,TRAVEL,TRAVEL_LODGING,TRAVEL,TRAVEL_LODGING,Personal Spending,Travel,-17.40,-103.515


ACTUAL AMOUNT
{'2023-04-01': -473.93, '2023-05-01': -2.21, '2023-07-01': -122.03, '2023-11-01': -75.0, '2024-01-01': -85.0, '2024-02-01': -299.22, '2024-05-01': -3605.55, '2024-07-01': -17.4}

MEDIAN AMOUNT
{'2023-04-01': -103.515, '2023-05-01': -103.515, '2023-07-01': -103.515, '2023-11-01': -103.515, '2024-01-01': -103.515, '2024-02-01': -103.515, '2024-05-01': -103.515, '2024-07-01': -103.515}

TRAVEL_OTHER_TRAVEL


,transaction_month,category_raw,subcategory_raw,category_updated,subcategory_updated,category,subcategory,amount,median_per_category
258,2024-07-01,TRAVEL,TRAVEL_OTHER_TRAVEL,TRAVEL,TRAVEL_OTHER_TRAVEL,Personal Spending,Travel,-16.8,-16.8


ACTUAL AMOUNT
{'2024-07-01': -16.8}

MEDIAN AMOUNT
{'2024-07-01': -16.8}

